In [ ]:
# Extract - Data Extraction from Files

# Install required dependencies
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!sudo python get-pip.py
!sudo pip install Django
!sudo pip install Pillow  # Image processing
!sudo pip install pyspark  # Running Spark

In [ ]:
# Extract - Setting up Django project

!django-admin startproject smartlibrary
%cd smartlibrary
!python manage.py runserver

In [ ]:
# Extract - Creating Django applications

!python manage.py startapp authorize
!python manage.py startapp home

In [ ]:
# Extract - Defining Django Data Models

from django.db import models

# User model
class User(models.Model):
    name = models.CharField(max_length=50, default='')
    email = models.EmailField()
    password = models.CharField(max_length=6, default='admin')

# Book model
class Book(models.Model):
    no = models.CharField(max_length=50, default='', verbose_name="Book ID")
    name = models.CharField(max_length=50, default='', verbose_name="Title")
    price = models.CharField(max_length=50, default='', verbose_name="Price")
    cover = models.ImageField(verbose_name="Cover", upload_to='upload', default='img/default.png')
    introduction = models.TextField(verbose_name="Description", default='', blank=True)
    url = models.URLField(verbose_name='URL', default='', blank=True)
    publish = models.CharField(verbose_name='Publisher', max_length=50, default='', blank=True)
    rating = models.CharField(verbose_name='Rating', max_length=5, default='0')

# Click model for tracking user interactions
class Hits(models.Model):
    userid = models.IntegerField(default=0)
    bookid = models.IntegerField(default=0)
    hitnum = models.IntegerField(default=0)

In [ ]:
# Load - Migrating database

!python manage.py makemigrations authorize
!python manage.py makemigrations home
!python manage.py migrate

In [ ]:
# Extract - Importing book data from uploaded file

from django.shortcuts import render, HttpResponse
from home.models import Book, Hits
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def importBookData(request):
    if request.method == 'POST':
        file = request.FILES.get('file', None)
        if not file:
            return HttpResponse('No File uploaded!')
        else:
            name = file.name
            handle_upload_file(name, file)
            return HttpResponse('success')

# Save uploaded file
def handle_upload_file(name, file):
    with open(name, 'wb') as destination:
        for chunk in file.chunks():
            destination.write(chunk)
    insertToSQL(name)

# Insert book data into database
def insertToSQL(fileName):
    with open(fileName, 'r') as txtfile:
        for line in txtfile.readlines():
            bookinfo = line.split(',')
            bk_entry = Book(no=bookinfo[0], name=bookinfo[1], rating=bookinfo[2],
                            price=bookinfo[3], publish=bookinfo[4], url=bookinfo[5])
            bk_entry.save()

In [ ]:
# Transform - Processing user click data

from pyspark.shell import sc
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("train hits data").getOrCreate()

ratings = []

# Read user click data from hits.txt and transform it into a structured format
with open("hits.txt", "r") as f:
    for line in f.readlines():
        user_id, book_id, hit_count = map(int, line.split(','))
        ratings.append((user_id, book_id, hit_count))

# Convert to Spark RDD format
rdd = sc.parallelize(ratings)
hits_rdd = rdd.map(lambda x: (x[0], x[1], x[2]))

# Train ALS recommendation model
model = ALS.trainImplicit(hits_rdd, rank=10, iterations=10, lambda_=0.01)

# Save the trained model
model.save(spark.sparkContext, 'recommendModel')

# Load the trained model and provide recommendations
def getRecommendByUserId(userid, rec_num):
    try:
        model = MatrixFactorizationModel.load(sc, 'recommendModel')
        recommendations = model.recommendProducts(userid, rec_num)
        print(recommendations)
    except Exception as e:
        print(f"Load model failed: {e}")

getRecommendByUserId(1, 5)

In [ ]:
# Load - Installing and using Redis

!wget http://download.redis.io/releases/redis-5.0.4.tar.gz
!tar xzf redis-5.0.4.tar.gz
%cd redis-5.0.4
!make
!src/redis-server  # Start Redis server
!src/redis-cli     # Start Redis client

In [ ]:
# Load - Storing recommendation results in Redis

import redis

# Initialize Redis connection
pool = redis.ConnectionPool(host='localhost', port=6379)
redis_client = redis.Redis(connection_pool=pool)

# Function to retrieve recommended books from Redis cache
def getRecommendBook(request):
    userid = request.GET.get('userid')
    recommend_books = redis_client.get(int(userid))
    book_list = str(recommend_books).split('|')
    books = [Book.objects.get(id=bk.split(',')[1]) for bk in book_list if bk]
    return render(request, 'home/recommend.html', {'books': books})

In [ ]:
# Load - Setting up Django user authentication

def register(request):
    if request.method == 'POST':
        name = request.POST.get('name')
        password = request.POST.get('password')
        userEntry = User(name=name, password=password)
        userEntry.save()
        return HttpResponseRedirect('/login')
    return render(request, 'auth/register.html')

def login(request):
    if request.method == 'POST':
        name = request.POST.get('name')
        password = request.POST.get('password')
        userEntry = User.objects.filter(name=name, password=password)
        if userEntry.exists():
            request.session['user'] = name
            return HttpResponseRedirect('/')
    return render(request, 'auth/login.html')

In [ ]:
# Load - Django URL routing

from django.urls import path
from home.views import index, getBookInfo, getRecommendBook, importBookData
from authorize.views import login, register, logout

urlpatterns = [
    path('', index, name='index'),
    path('login/', login, name='login'),
    path('register/', register, name='register'),
    path('getbookinfo/', getBookInfo, name='getbookinfo'),
    path('bookpush/', getRecommendBook, name='getrecommendbook'),
    path('upload/', importBookData, name='importBookData'),
]